In [1]:
# Import libraries

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from utils.ML_helper import cargar_videos_y_etiquetas

In [2]:
cargar_videos_y_etiquetas

<function utils.ML_helper.cargar_videos_y_etiquetas(ruta_base, clases, num_frames=30)>

# Build CNN Model

In [3]:
clases = ['Cometa', 'Ecuador_celeste', 'Meteoroide', 'Meteoro', 'Zodiaco']
ruta_base = "Dataset"

# Upload the dataset
imagenes, etiquetas = cargar_videos_y_etiquetas(ruta_base, clases)
x_train, x_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam

# Definir la arquitectura CNN + LSTM
modelo = Sequential([
    # Extraer características con CNN
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', input_shape=(None, 200, 200, 3))),
    TimeDistributed(MaxPooling2D((2, 2))),

    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),

    TimeDistributed(Conv2D(128, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),

    TimeDistributed(Conv2D(256, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),

    TimeDistributed(Flatten()),  # Aplanar cada frame

    # LSTM para aprender la relación entre los frames
    LSTM(128, return_sequences=False),  

    # Capa totalmente conectada
    Dense(128, activation='relu'),
    Dropout(0.5),

    # Capa de salida
    Dense(len(clases), activation='softmax')
])

modelo.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

e:\UdeA\Aprendizaje-estadistico\LSC_Astronophysics\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
modelo.fit(x_train, y_train, epochs=10, batch_size=8, validation_data=(x_test, y_test))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 91s 91s/step - accuracy: 0.2500 - loss: 1.6094 - val_accuracy: 0.0000e+00 - val_loss: 1.6110
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 56s 56s/step - accuracy: 0.2500 - loss: 1.6090 - val_accuracy: 0.0000e+00 - val_loss: 1.6126
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 60s 60s/step - accuracy: 0.2500 - loss: 1.6086 - val_accuracy: 0.0000e+00 - val_loss: 1.6142
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 51s 51s/step - accuracy: 0.2500 - loss: 1.6082 - val_accuracy: 0.0000e+00 - val_loss: 1.6158
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 52s 52s/step - accuracy: 0.2500 - loss: 1.6078 - val_accuracy: 0.0000e+00 - val_loss: 1.6174
Epoch 6/10


In [ ]:
# Convolutional Neural Network (CNN)
modelo = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(clases), activation='softmax')
])

modelo.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
tensorboard = TensorBoard(log_dir="logs")
modelo.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test), callbacks=[tensorboard])

In [ ]:
# Make predictions
y_pred_prob = modelo.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class indices

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=clases, yticklabels=clases)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Evaluation and metrics
resultados = modelo.evaluate(x_test, y_test)
print(f"Precisión: {resultados[1] * 100:.2f}%")

# Save the model
modelo.save('modelo_lsc2.h5')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils.class_weight import compute_class_weight
from utils.ML_helper import cargar_datos_y_etiquetas

# Define dataset path and classes
ruta_base = "Dataset"
clases = ['Cometa', 'Ecuador_celeste', 'Meteoroide', 'Meteoro', 'Zodiaco']

# Load dataset
imagenes, etiquetas = cargar_datos_y_etiquetas(ruta_base, clases)

# Split dataset into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Ensure y_train is a 1D integer array
y_train_flat = np.ravel(y_train).astype(int)

# Compute Class Weights to Balance Dataset
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train_flat), 
    y=y_train_flat
)
class_weights_dict = {i: class_weights[i] for i in np.unique(y_train_flat)}

# Define CNN Model
modelo = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(200, 200, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(clases), activation='softmax')
])

# Compile the model
modelo.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Model
tensorboard = TensorBoard(log_dir="logs")
modelo.fit(
    x_train, y_train_flat,
    epochs=30,  # Increased epochs for better learning
    batch_size=32,
    validation_data=(x_test, y_test),
    class_weight=class_weights_dict,  # Balanced class training
    callbacks=[tensorboard]
)

# Save the Model
modelo.save('modelo_lsc3.h5')

print("Model training complete and saved!")


In [ ]:
# Save the Model
modelo.save('modelo_lsc3.h5')

print("Model training complete and saved!")

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for test set

train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
test_generator = test_datagen.flow(x_test, y_test, batch_size=32)


In [ ]:
from tensorflow.keras.layers import BatchNormalization

modelo = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(200, 200, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(clases), activation='softmax')
])

modelo.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Ensure y_train is a 1D integer array with the correct length
y_train_flat = np.array(y_train).reshape(-1)  # Ensure it's 1D and has the correct length
print("Expected samples:", x_train.shape[0])
print("Actual y_train samples:", y_train_flat.shape[0])


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ensure y_train is a 1D integer array
y_train_flat = np.ravel(y_train).astype(int)  # Convert to 1D and ensure integer labels

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_flat), y=y_train_flat)

# Convert to dictionary format required by model.fit()
class_weights_dict = {cls: weight for cls, weight in zip(np.unique(y_train_flat), class_weights)}

# Train the model using class weights
modelo.fit(x_train, y_train_flat, epochs=10, batch_size=32, validation_data=(x_test, y_test),
           class_weight=class_weights_dict, callbacks=[tensorboard])


# Test Real Time Model

In [ ]:
# Load the trained model
model = tf.keras.models.load_model("modelo_lsc2.h5")

# Define class labels (Ensure this matches your training classes)
clases = ['Cometa', 'Ecuador_celeste', 'Meteoroide', 'Meteoro', 'Zodiaco']  # Update this with your classes

# Initialize MediaPipe for hand tracking
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Start webcam capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB and process hand detection
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    predictions = []  # Store predictions for multiple hands

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get bounding box around the hand
            h, w, _ = frame.shape
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * w)
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * w)
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * h)
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * h)

            # Extract and preprocess hand image
            hand_img = frame[y_min:y_max, x_min:x_max]
            if hand_img.size > 0:
                hand_img = cv2.resize(hand_img, (200, 200))
                hand_img = hand_img.astype('float32') / 255.0  # Normalize
                hand_img = np.expand_dims(hand_img, axis=0)  # Add batch dimension

                # Predict the class
                prediction = model.predict(hand_img)
                class_index = np.argmax(prediction)
                predictions.append(clases[class_index])

    # Display only the most confident prediction
    if predictions:
        most_common_prediction = max(set(predictions), key=predictions.count)
        cv2.putText(frame, f"Prediction: {most_common_prediction}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the webcam feed with predictions
    cv2.imshow("Real-Time Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("modelo_lsc3.h5")

# Define class labels (Ensure this matches your training classes)
clases = ['Cometa', 'Ecuador_celeste', 'Meteoroide', 'Meteoro', 'Zodiaco']  # Update this with your classes

# Initialize MediaPipe for hand tracking
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Load the video file
video_path = "Dataset/Meteoroide/14.mp4"  # Change this to your uploaded video filename
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output processed video
output_path = "output_video.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB and process hand detection
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    predictions = []  # Store predictions for multiple hands

    if result.multi_hand_landmarks:
        x_min, y_min, x_max, y_max = frame.shape[1], frame.shape[0], 0, 0
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get bounding box around all detected hands
            h, w, _ = frame.shape
            x_min = min(x_min, int(min([landmark.x for landmark in hand_landmarks.landmark]) * w))
            x_max = max(x_max, int(max([landmark.x for landmark in hand_landmarks.landmark]) * w))
            y_min = min(y_min, int(min([landmark.y for landmark in hand_landmarks.landmark]) * h))
            y_max = max(y_max, int(max([landmark.y for landmark in hand_landmarks.landmark]) * h))

        # Extract and preprocess hand image
        hand_img = frame[y_min:y_max, x_min:x_max]
        if hand_img.size > 0:
            hand_img = cv2.resize(hand_img, (200, 200))
            hand_img = hand_img.astype('float32') / 255.0  # Normalize
            hand_img = np.expand_dims(hand_img, axis=0)  # Add batch dimension

            # Predict the class
            prediction = model.predict(hand_img)
            class_index = np.argmax(prediction)
            predicted_label = clases[class_index]

            # Display the prediction
            cv2.putText(frame, f"Prediction: {predicted_label}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Write the processed frame to the output video
    out.write(frame)

    # Show the processed video frame
    cv2.imshow("Video Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Processed video saved as {output_path}")
